# Adicionando Funções externad utilizando Langchain

Langchain é um framework para criação de aplicações de IA e naturalmente ele possui ferramentas para facilitar a criação de funções externas para serem passadas a api da OpenAI. Para a utilização do LangChain, será necessário entendermos brevemente uma outra biblioteca de Python chamada pydantic, uma biblioteca para validação de dados que facilita a construção de estruturas de dados mais robustas.

In [2]:
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Optional
from enum import Enum

class UnidadeEnum(str, Enum):
  celsius = 'celsius'
  fahrenheit = 'fahrenheit'
  

class ObterTemperaturaAtua(BaseModel):
  """Obtém a temperatura atual de uma determinada localidade"""
  local: str = Field(description='O nome da cidade', examples=['São Paulo', 'Porto Alegre'])
  unidade: Optional[UnidadeEnum]
  

In [3]:
from langchain_core.utils.function_calling import convert_to_openai_function

tool_temperatura = convert_to_openai_function(ObterTemperaturaAtua)
tool_temperatura

{'name': 'ObterTemperaturaAtua',
 'description': 'Obtém a temperatura atual de uma determinada localidade',
 'parameters': {'type': 'object',
  'properties': {'local': {'description': 'O nome da cidade',
    'examples': ['São Paulo', 'Porto Alegre'],
    'type': 'string'},
   'unidade': {'description': 'An enumeration.',
    'enum': ['celsius', 'fahrenheit'],
    'type': 'string'}},
  'required': ['local']}}

## Adicionando função externa utilizando LangChain

Agora que já sabemos criar funções que os modelos de llm entendam, podemos passar essas funções para os modelos de llm através da biblioteca langchain. Para isso temos duas formas, podemos utilizar o parâmetro functions ao chamar o métdoso chat_models.

In [5]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI()

response = chat.invoke('Qual é a temperatura de Sao Paulo', functions=[tool_temperatura])
response

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"local":"São Paulo"}', 'name': 'ObterTemperaturaAtua'}}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 104, 'total_tokens': 127, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-d8f56aa9-f76a-4d0b-9612-d6bc4b6f3c81-0')

Ou podemos dar um bind e criar um novo componente de chat_model que terá acesso a função sempre que for chamado o invoke.
Nestes dois casos o modelo se comportará com o parâmetro "auto" de cahamento de função ou seja, ele chamará a função quando necessitar, caso contrário se comportará como um modelo de linguagem normal.

In [8]:
chat= ChatOpenAI()

chat_com_func = chat.bind(functions=[tool_temperatura])


response = chat_com_func.invoke('Qual é a temperatura de Sao Paulo')
response

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"local":"São Paulo"}', 'name': 'ObterTemperaturaAtua'}}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 104, 'total_tokens': 127, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-415f3df0-8b54-4fe7-afe0-9080028c3798-0')

Podemos obrigar o modelo a sempre chamar uma função da seguinte forma:

In [9]:

response = chat.invoke(
  'Qual é a temperatura de Sao Paulo', 
  functions=[tool_temperatura],
  function_call={'name': 'ObterTemperaturaAtua'}
  )
response

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"local":"São Paulo"}', 'name': 'ObterTemperaturaAtua'}}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 119, 'total_tokens': 127, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-badbad67-7c95-4ae9-b3bb-062413c047df-0')

In [10]:
response = chat.invoke(
  'Olá', 
  functions=[tool_temperatura],
  function_call={'name': 'ObterTemperaturaAtua'}
  )
response

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"local":"São Paulo"}', 'name': 'ObterTemperaturaAtua'}}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 114, 'total_tokens': 122, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-7049fdeb-1a87-454b-b4ba-93dbf8e2baab-0')

## Adicionando a uma chain

Podemos adicionar agora este modelo com funções a um prompt e criar uma chain.

In [13]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
  ('system', 'Você é um assistente amigável chamado Jeremias'),
  ('user', '{input}')
])

chain = prompt | chat.bind(functions=[tool_temperatura])

In [14]:
chain.invoke({'input': 'Olá'})

AIMessage(content='Olá! Como posso ajudar você hoje?', response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 113, 'total_tokens': 126, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2907a509-2fb3-4b48-a685-d6b9aec6fa48-0')

In [15]:
chain.invoke({'input': 'Qual a temperatura em Itanhaém'})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"local":"Itanhaém"}', 'name': 'ObterTemperaturaAtua'}}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 119, 'total_tokens': 143, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-edc0ef2b-b907-4f37-9d50-31031d2470a1-0')

# Um exemplo mais interessante 

Indo para um exemplo um pouco mais complexo. Digamos que temos um chatbot e queremos fazer um toreamento para os setores de interesse, no nosso caso atendimento_cliente, dúvidas_alunos, vendas, spam. A primeira etapa é criar um modelo que entenda a solicitação do cliente e direcione para o setor certo. Nas próximas etapas o atendimento pode ser continuado por pessoas ou por agentes especializados para as tarefas do setor. Retiramos as seguintes mensagens de email rececidos pela equipe de atendimento e vamos tentar criar um direcionamento para elas:

In [1]:
duvidas = [
  "Bom dia, gostaria de saber se ha um certificado final para cada trilha ou se os certificados são somente para todos os cursos",
  "In Esy, Amazon, eBay, Shopify, Pinterest+SEO +II = high sales results",
  "Boa tarde, esto iniciando hoje  e estou perdido. Tenho vairos objtivos. Não sei nada de programação, exceto que utilizo nos estudos, Podem meu ajudar e me tornar um profissional em programação?",
  "Bom, dia. Não estou conseguindo tirar os certificados dos cursos que conclui.",
  "Bom dia. Não encontrei no site o preço de um curso avulso. Saberiam me informar?",
]

In [2]:
from enum import Enum 
from langchain.pydantic_v1 import BaseModel, Field 

In [5]:
class SetorEnum(str, Enum):
  atendimento_cliente = 'atendimento_cliente'
  duvidas_aluno = 'duvidas_aluno'
  vendas = 'vendas'
  spam = 'spam'
  
class DirecionaSetorResponsavel(BaseModel):
  """Direciona a dúvida de um cliente ou aluno da escola de programação Asimov para o setor responsável"""
  setor: SetorEnum

In [3]:
from langchain_core.utils.function_calling import convert_to_openai_function


In [6]:
tool_direcionamento = convert_to_openai_function(DirecionaSetorResponsavel)
tool_direcionamento

{'name': 'DirecionaSetorResponsavel',
 'description': 'Direciona a dúvida de um cliente ou aluno da escola de programação Asimov para o setor responsável',
 'parameters': {'type': 'object',
  'properties': {'setor': {'description': 'An enumeration.',
    'enum': ['atendimento_cliente', 'duvidas_aluno', 'vendas', 'spam'],
    'type': 'string'}},
  'required': ['setor']}}

In [10]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

In [12]:
prompt = ChatPromptTemplate.from_messages([
    ('system', 'Pense com cuidado ao categorizar o texto conforme as instruções'),
    ('user', '{input}')
])

chat = ChatOpenAI()

chain = (
          prompt 
          | chat.bind(functions=[tool_direcionamento], function_call={'name': 'DirecionaSetorResponsavel'})
          | JsonOutputFunctionsParser()
        )

In [17]:
duvida = duvidas[4]
resposta = chain.invoke({'input': duvida})

print('Dúvida', duvida)
print('Resposta', resposta)

Dúvida Bom dia. Não encontrei no site o preço de um curso avulso. Saberiam me informar?
Resposta {'setor': 'atendimento_cliente'}


Neste caso gostaríamos que a dúvida fosse direcionada para vendas. Podemos melhorar nosso prompt para aumentar a chance do modelo responder como gostaríamos.

In [18]:
system_message = '''Pense com cuidado ao categorizar o texto conforme as instruções.
    Questões relacionadas a dúvidas de preço, sobre o produto, como funciona devem ser direcionados para "vendas".
    Questões relacionadas a conta, acesso a plataforma, a cancelamento e renovação de assinatura devem ser direcionadas para "atendimento_cliente".
    Questões relacinadas a dúvidas técnicas de programação, conteúdos da plataforma ou tecnologias na área da programação, devem ser direcionadas para "duvidas_aluno".
    Mensagens suspeitas, em outras linguas que não português, contendo links devem ser direcionadas para "spam"
     '''

prompt = ChatPromptTemplate.from_messages([
    ('system', system_message),
    ('user', '{input}')
])

chat = ChatOpenAI()

chain = (
          prompt 
          | chat.bind(functions=[tool_direcionamento], function_call={'name': 'DirecionaSetorResponsavel'})
          | JsonOutputFunctionsParser()
        )

In [20]:
duvida = duvidas[1]
resposta = chain.invoke({'input': duvida})

print('Dúvida', duvida)
print('Resposta', resposta)

Dúvida In Esy, Amazon, eBay, Shopify, Pinterest+SEO +II = high sales results
Resposta {'setor': 'spam'}
